In [2]:
import streamlit as st
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os
import openai
from openai import OpenAI

load_dotenv()
#openai.api_key = st.secrets.get("OPENAI_API_KEY", os.getenv("OPENAI_API_KEY"))

#import cmdstanpy
#cmdstanpy.install_cmdstan()


# Definir las fechas globalmente
end_date = datetime.now()
start_date = end_date - timedelta(days=5*365)

In [3]:
top_stocks = 'top_28_growth_stocks.csv'
df = pd.read_csv(top_stocks)

In [6]:
top_stocks_tickers = df['Ticker'].tolist()
top_stocks_names = df['name'].tolist()
print("top_stocks_tickers: ")
print(top_stocks_tickers)
print("top_stocks_names: ")
print(top_stocks_names)


top_stocks_tickers: 
['CWEN', 'CODQL', 'GNK', 'AVTR', 'FCN', 'MRTN', 'XYL', 'PDEX', 'ETR', 'SPRS', 'HURN', 'RSG', 'CWST', 'CTS', 'TRMB', 'ANSS', 'ADEA', 'NI', 'ACCO', 'ALE', 'CXT', 'HEI', 'MSA', 'RBC', 'MWA', 'ALB', 'LNG', 'CVLG']
top_stocks_names: 
['CLEARWAY ENERGY, INC.', 'CORONADO GLOBAL RESOURCES INC.', 'GENCO SHIPPING & TRADING LTD', 'AVANTOR, INC.', 'FTI CONSULTING, INC', 'MARTEN TRANSPORT LTD', 'XYLEM INC.', 'PRO DEX INC', 'ENTERGY CORP /DE/', 'SURGE COMPONENTS INC', 'HURON CONSULTING GROUP INC.', 'REPUBLIC SERVICES, INC.', 'CASELLA WASTE SYSTEMS INC', 'CTS CORP', 'TRIMBLE INC.', 'ANSYS INC', 'XPERI HOLDING CORP', 'NISOURCE INC.', 'ACCO BRANDS CORP', 'ALLETE INC', 'CRANE CO /DE/', 'HEICO CORP', 'MSA SAFETY INC', 'RBC BEARINGS INC', 'MUELLER WATER PRODUCTS, INC.', 'ALBEMARLE CORP', 'CHENIERE ENERGY INC', 'COVENANT LOGISTICS GROUP, INC.']


In [ ]:
import yfinance as yf

# Comprobar si los tickers existen en yfinance
for ticker, name in zip(top_stocks_tickers, top_stocks_names):
    try:
        data = yf.Ticker(ticker).history(period="1d")
        if data.empty:
            print(f"Ticker NO válido o sin datos: {ticker} ({name})")
    except Exception as e:
        print(f"Error con {ticker} ({name}): {e}")

Ticker válido: CWEN (CLEARWAY ENERGY, INC.)
Ticker válido: CODQL (CORONADO GLOBAL RESOURCES INC.)
Ticker válido: GNK (GENCO SHIPPING & TRADING LTD)
Ticker válido: AVTR (AVANTOR, INC.)
Ticker válido: FCN (FTI CONSULTING, INC)
Ticker válido: MRTN (MARTEN TRANSPORT LTD)
Ticker válido: XYL (XYLEM INC.)
Ticker válido: PDEX (PRO DEX INC)
Ticker válido: ETR (ENTERGY CORP /DE/)
Ticker válido: SPRS (SURGE COMPONENTS INC)
Ticker válido: HURN (HURON CONSULTING GROUP INC.)
Ticker válido: RSG (REPUBLIC SERVICES, INC.)
Ticker válido: CWST (CASELLA WASTE SYSTEMS INC)
Ticker válido: CTS (CTS CORP)
Ticker válido: TRMB (TRIMBLE INC.)
Ticker válido: ANSS (ANSYS INC)
Ticker válido: ADEA (XPERI HOLDING CORP)
Ticker válido: NI (NISOURCE INC.)
Ticker válido: ACCO (ACCO BRANDS CORP)
Ticker válido: ALE (ALLETE INC)
Ticker válido: CXT (CRANE CO /DE/)
Ticker válido: HEI (HEICO CORP)
Ticker válido: MSA (MSA SAFETY INC)
Ticker válido: RBC (RBC BEARINGS INC)
Ticker válido: MWA (MUELLER WATER PRODUCTS, INC.)
Ticker 

In [ ]:
def cargar_datos():
    symbols = {
    "Apple": "AAPL", "Microsoft": "MSFT", "Amazon": "AMZN", "Alphabet (Google)": "GOOGL", "Tesla": "TSLA",
    "Berkshire Hathaway": "BRK-B", "Johnson & Johnson": "JNJ", "Exxon Mobil": "XOM", "JPMorgan Chase": "JPM", "Visa": "V",
    "Nestlé": "NESN.SW", "Roche": "ROG.SW", "Samsung Electronics": "005930.KS", "Toyota": "7203.T", "Sony": "6758.T",
    "Alibaba": "BABA", "Tencent": "0700.HK", "HSBC": "HSBA.L", "BP": "BP.L", "Shell": "SHEL.L",
    "Unilever": "ULVR.L", "LVMH": "MC.PA", "TotalEnergies": "TTE.PA", "Siemens": "SIE.DE", "Volkswagen": "VOW3.DE",
    "SAP": "SAP.DE", "Banco Santander": "SAN.MC", "BBVA": "BBVA.MC", "Petrobras": "PBR", "Vale": "VALE",
    "ICICI Bank": "IBN", "Reliance Industries": "RELIANCE.BO", "Infosys": "INFY", "Ping An Insurance": "2318.HK",
    "China Mobile": "0941.HK", "Rio Tinto": "RIO.L", "BHP Group": "BHP.AX", "Commonwealth Bank": "CBA.AX",
    "CSL Limited": "CSL.AX", "Novo Nordisk": "NOVO-B.CO", "AstraZeneca": "AZN.L", "Adidas": "ADS.DE",
    "Heineken": "HEIA.AS", "Philips": "PHIA.AS", "Enel": "ENEL.MI", "Ferrari": "RACE", "Saudi Aramco": "2222.SR",
    "Tencent Music": "TME", "Meta Platforms (Facebook)": "META", "Procter & Gamble": "PG", "Coca-Cola": "KO",
    "PepsiCo": "PEP", "McDonald's": "MCD", "Walmart": "WMT", "Costco": "COST", "Intel": "INTC",
    "AMD": "AMD", "NVIDIA": "NVDA", "Qualcomm": "QCOM", "Broadcom": "AVGO", "Texas Instruments": "TXN",
    "IBM": "IBM", "Oracle": "ORCL", "Salesforce": "CRM", "Adobe": "ADBE", "Netflix": "NFLX",
    "AT&T": "T", "Verizon": "VZ", "T-Mobile": "TMUS", "Pfizer": "PFE", "Moderna": "MRNA",
    "Merck": "MRK", "Bristol-Myers Squibb": "BMY", "Amgen": "AMGN", "Gilead Sciences": "GILD", "Eli Lilly": "LLY",
    "Chevron": "CVX", "ConocoPhillips": "COP", "Schlumberger": "SLB", "Halliburton": "HAL", "Marathon Oil": "MRO",
    "Lockheed Martin": "LMT", "Northrop Grumman": "NOC", "Raytheon Technologies": "RTX", "Boeing": "BA",
    "General Electric": "GE", "Honeywell": "HON", "3M": "MMM", "Caterpillar": "CAT", "Deere & Company": "DE",
    "Starbucks": "SBUX", "Nike": "NKE", "Lululemon": "LULU", "Estee Lauder": "EL", "Domino's Pizza": "DPZ",
    "Booking Holdings": "BKNG", "American Express": "AXP", "Mastercard": "MA", "PayPal": "PYPL"
        # Puedes agregar más compañías aquí para tu MVP
    }
    data = pd.DataFrame()
    for company, symbol in symbols.items():
        df = yf.download(symbol, start=start_date, end=end_date)['Close']
        if not df.empty:
            data[company] = df
        else:
            print(f"No se pudieron descargar los datos para {company} ({symbol}).")
    return data.dropna()

data = cargar_datos()